## Tuning Combined Model with Random Search

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

# Keras-related modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from keras_tuner import HyperModel
from sklearn.model_selection import train_test_split

2022-07-24 21:41:33.643549: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### 1. Load data from file and prepare to be used for tuning

Load the EoS and the $M-R$ curves from processed 3P/4L data files, then merge 3P and 4L data.

In [2]:
# Load the un-normalized data
R_3p = np.loadtxt('../data/mrc_3p.txt')
P_3p = np.loadtxt('../data/eos_3p.txt')
R_4l = np.loadtxt('../data/mrc_4l.txt')
P_4l = np.loadtxt('../data/eos_4l.txt')
R_td = np.loadtxt('../data/mrc_td.txt')
P_td = np.loadtxt('../data/eos_td.txt')

# Merge 3P and 4L data
R = np.concatenate((R_3p, R_4l, R_td))
P = np.concatenate((P_3p, P_4l, P_td))

Next, normalize the data, then shuffle the data and perform a train-test split as 80-20.

In [3]:
# Normalize the data
r = (R - np.min(R)) / (np.max(R) - np.min(R))
p = (P - np.min(P)) / (np.max(P) - np.min(P))

# Perform train-test split as 80-20
x_tr, x_ts, y_tr, y_ts = train_test_split(r, p, test_size=0.2, shuffle=True, random_state=41)

### 2. Construct the search space

We seek to find the values of hyperparameters that optimizes the DNN for performance and accuracy. The choices of hyperparameters include number of layers, number of neurons/units in each layer, the activation functions on the inner layers and the output layer. 

Even with less number of hyperparameters, the search space can be large and the process may take a while to complete, depending on their ranges we choose. Therefore, to expedite the tuning, we fix some hyperparameters and search over the others.

In [4]:
# Define the search space
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape    
        
    def build(self, hp):
        model = keras.Sequential()
        # Tune the number of layers
        for i in range(hp.Int('num_layers', 1, 4)):
            model.add(
                layers.Dense(
                    # Tune number of units separately
                    units=hp.Int(f"units_{i}", min_value=200, max_value=800, step=100),
                    activation='relu'
                )
            )
        model.add(layers.Dense(100, activation='linear'))
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model

In [5]:
# Initialize the input shape
input_shape = (x_tr.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

### 3. Initialize search parameters

Set values of search parameters and create an early-stopping callback. 

In [6]:
# Initialize the search
tuner = kt.RandomSearch(
    hypermodel,                # Pass the hypermodel object
    objective='val_loss',      # Quantity to monitor during tuning
    seed=42,                   # Set reproducibility of randomness
    max_trials=5000,           # Max number of trials with different hyperparameters
    executions_per_trial=1,    # Number of repeated trials with same hyperparameters
    directory="random_search", # Set directory to store search results
    project_name="all",        # Set the subdirectory name
    overwrite=True             # Choose if previous search results should be ignored
)
# Set up callback for early stopping 
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1.0e-6, patience=10)

# Print the summary of search space
tuner.search_space_summary()

Search space summary
Default search space size: 2
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 800, 'step': 100, 'sampling': None}


2022-07-24 21:41:36.806037: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-24 21:41:36.806743: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-24 21:41:36.841836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 21:41:36.841979: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 with Max-Q Design computeCapability: 7.5
coreClock: 1.245GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 104.34GiB/s
2022-07-24 21:41:36.841995: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-24 21:41:36.843045: I tensorflow/stream_ex

Now begin tuning the network.

In [7]:
tuner.search(x_tr, y_tr, batch_size=1024, epochs=5000, validation_data=(x_ts, y_ts), \
                callbacks=[stop_early], verbose=2)

Trial 798 Complete [00h 00m 11s]
val_loss: 0.0006508337683044374

Best val_loss So Far: 0.0006201856303960085
Total elapsed time: 02h 33m 38s

Search: Running Trial #799

Hyperparameter    |Value             |Best Value So Far 
num_layers        |4                 |4                 
units_0           |200               |300               
units_1           |200               |500               
units_2           |700               |800               
units_3           |600               |300               

Epoch 1/5000
21/21 - 0s - loss: 0.0025 - val_loss: 9.0904e-04
Epoch 2/5000
21/21 - 0s - loss: 9.1014e-04 - val_loss: 8.7794e-04
Epoch 3/5000
21/21 - 0s - loss: 8.8440e-04 - val_loss: 8.5848e-04
Epoch 4/5000
21/21 - 0s - loss: 8.6609e-04 - val_loss: 8.2524e-04
Epoch 5/5000
21/21 - 0s - loss: 8.5114e-04 - val_loss: 9.0723e-04
Epoch 6/5000
21/21 - 0s - loss: 8.5845e-04 - val_loss: 8.2101e-04
Epoch 7/5000
21/21 - 0s - loss: 8.4090e-04 - val_loss: 8.1263e-04
Epoch 8/5000
21/21 - 0s - lo

KeyboardInterrupt: 

### 4. Publish search results

Print the first few (5-10) top models and then pick the best model by hand.

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
# Get the top model
models = tuner.get_best_models(num_models=10)
best_model = models[0]

# Build the best model
best_model.build(input_shape=(None, 100))

# Show the best model
best_model.summary()

Evaluate the best model and note the order of magnitude of the loss function for reference.

In [ ]:
loss = best_model.evaluate(x_ts, y_ts, verbose=0)
print("Loss = {:.4e}".format(loss))

### 5. Save the best model to file

Tuning a network is computationally expensive. Besides, the results are not reproducible because of the stochastic nature of this mechanism. Therefore, we tune a network only once for a given data set and do not repeat it unless either the input shape or the data set itself has changed.

Write the best model to file so that it can be loaded directly without repeating the search.

In [ ]:
best_model.save("../output/model_all.h5")